# ML to find radial velocities using GALAXIA sim - need to change dataset

In [ ]:
%matplotlib inline


import matplotlib
import matplotlib.colors as colors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
import sys
import gzip
import matplotlib.gridspec as gridspec
from scipy.stats import norm

matplotlib.rcParams.update({'font.family':'cmr10','font.size': 13})
matplotlib.rcParams['axes.unicode_minus']=False
matplotlib.rcParams['axes.labelsize']=15
plt.rcParams['figure.figsize']=(4,4)
plt.rcParams['figure.dpi'] = 80
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.rm'] = 'serif'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True

In [ ]:
data = np.load('/tigress/ljchang/DataXGaia/data/galaxia_mock/processed_stars_reshuffled_with_pos_vel.npy')

In [ ]:
data_cols = ['source_id', 'l', 'b', 'ra', 'dec', 'parallax', 'parallax_error', 
             'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'radial_velocity',
             'photo_g_mean_mag', 'photo_bp_mean_mag', 'photo_rp_mean_mag',
             'x','y','z','vx','vy','vz','r','phi','theta','vr','vphi','vtheta']
#could train on radial_velocity_error

In [ ]:
data = pd.DataFrame(data, columns=data_cols)

In [ ]:
data.shape

In [ ]:
data.head()


In [ ]:
#data = data.drop(data[data.AccretedLabel > 0.0].index)

indices_to_drop = np.load('data_indices_error_lt100_50k.npy')
print(np.shape(indices_to_drop))
print(indices_to_drop[1:10])

data = data.drop(data.index[indices_to_drop])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d

Remove 62,035 stars labeled as accreted:

In [ ]:
print(data.parallax[data.parallax < 0])

Remove 484569 stars with radial velocity greater than 100:

In [ ]:
#data = data.drop(data[data.radial_velocity < -100.0].index)

In [ ]:
# What columns to use?
use_cols = ['l', 'b','parallax','pmra','pmdec']
# Make the design matrix
X = data[use_cols].values
Y = (data['radial_velocity']).values
#Y = (Y - np.mean(Y))/(np.std(Y))

In [ ]:
print(X.shape)
print(Y.shape)

I think this is giving me a negative output because it's summing over y_train, which is not the number of stars, but is instead the radial velocities. 

In [ ]:
x_train, X_test, Y_train, y_test = train_test_split(X[0:500000], Y[0:500000], #44590
                                                    test_size = 0.2,
                                                    random_state=42)
#0:497448

In [ ]:
#np.save('truth_values_nontransformed',np.hstack((X_test,y_test.reshape(-1, 1))))

In [ ]:
SS = StandardScaler()

x_train_save = x_train
X_val_save = x_train_save[320001:400000,:]
Y_train_save = Y_train
X_test_save = X_test
#max_x_train_cols = np.max(x_train)

#x_train = np.divide(x_train, max_x_train_cols)
#X_test = np.divide(X_test, max_x_train_cols)

#Y_train_max = np.max(Y_train)
#Y_train = Y_train.reshape(-1, 1)/Y_train_max
#y_test = y_test.reshape(-1, 1)/Y_train_max

x_train = SS.fit_transform(x_train)
# Now scale the test data using the same mean and standard deviation as found with the training data

X_test = SS.transform(X_test)

SS_rv = StandardScaler()
Y_train = SS_rv.fit_transform(Y_train.reshape(-1, 1))
y_test = SS_rv.transform(y_test.reshape(-1, 1))



In [ ]:
Y_train = Y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
print(np.shape(Y_train))
print(np.shape(y_test))

In [ ]:
X_train = x_train[0:320000,:] #318336
y_train = Y_train[0:320000]
X_val = x_train[320001:400000,:]#397927
y_val = Y_train[320001:400000]
#need to split for validation
y_low = -700
y_high = 700

In [ ]:
print(X_train.shape)
print(y_train.shape)
print()
print(X_test.shape)
print(y_test.shape)
print()
print(X_val.shape)
print(y_val.shape)

In [ ]:
print(X_val[0:5,0])

In [ ]:
#trying to add some weights to training data
#might weight the mean, but no weights on confidence model? (something to think about)
vr_real = Y_train_save
counts, bins = np.histogram(vr_real,bins=np.linspace(-700,700,51))
bin_centers = (bins[1:]+bins[:-1])/2
interp_func  = interp1d(bin_centers,(counts).astype('float'))
inv_weights = interp_func(vr_real)
weights = 1/inv_weights
weights = np.log(weights)
weights = weights - np.min(weights)+1

weights_train = weights[0:320000]
weights_val = weights[320001:400000]
weights_train = weights_train.reshape(-1, 1)
weights_val = weights_val.reshape(-1, 1)
print(np.shape(weights_train))

#visualize check data
for i, col in enumerate(use_cols):
    plt.figure(figsize=(6, 3))
    plt.subplot(1,2,1)
    plt.hist(data[col], bins=100)
    plt.yscale('log')
    plt.title('Data')
    
    plt.subplot(1,2,2)
    plt.hist(X_train[:, i], bins=100, alpha=0.5)
    plt.hist(X_test[:, i], bins=100, alpha=0.5)
    plt.yscale('log')
    plt.title('Scaled')
    
    plt.suptitle(col, y=1.01, fontsize=16)
    
    plt.tight_layout()

In [ ]:
plt.hist(y_train, bins=100, range=(-5,5), histtype='step', color= 'blue', label = 'vlos', alpha = 0.5, density = True)
plt.hist(-y_train, bins=100, range=(-5,5), histtype='step', color= 'magenta',  label = '-vlos', alpha = 0.5, density = True)
plt.hist(y_train, bins=100, weights = weights_train, range=(-5,5), histtype='step', color= 'orange', label = 'vlos weighted', density = True)
plt.hist(-y_train, bins=100, weights = weights_train, range=(-5,5), histtype='step', color= 'red', label = '-vlos weighted', density = True)
plt.legend(bbox_to_anchor=(1.05, 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
from keras import backend as K
from keras import callbacks as callbacks
global index 

In [ ]:
def LikelihoodLossFunction(y_true, y_pred):
    # shape of y_pred should be (nsamples, 2)
    # the first column should be the mean of the prediction
    # the second column is the confidence (number of standard deviations)
#     print y_true.shape
#     print y_pred.shape
    SIGMA = K.abs(y_pred[:, 1]) + 1e-6
    LOC = y_pred[:, 0]
    X = y_true[:, 0]
    weights = y_true[:,1]
    ARG = K.square(X - LOC) / (2 * K.square(SIGMA))
    PREFACT = K.log(K.pow(2 * np.pi * K.square(SIGMA), -0.5))
    return K.mean((ARG - PREFACT) * weights)


In [ ]:
def ConstantLikelihoodLossFunction(y_true, y_pred):
    # shape of y_pred should be (nsamples, 2)
    # the first column should be the mean of the prediction
    # the second column is the confidence (number of standard deviations)
#     print y_true.shape
#     print y_pred.shape
    LOC = y_pred[:, 0]
    X = y_true[:, 0]
    weights = y_true[:,1]
    ARG = K.square(X - LOC) / (2.0)
    PREFACT = K.log(K.pow(2 * np.pi, -0.5))
    return K.mean((ARG - PREFACT) * weights)


# Two network technique to calculate the error

In [ ]:
from keras.layers import Input, Dense, Lambda, Concatenate, Dropout
from keras.models import Model
from keras.utils import plot_model
from IPython.display import clear_output
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [ ]:
y_train = np.hstack([y_train, weights_train])
y_val = np.hstack([y_val, weights_val])

In [ ]:
# del(inputs, MeanEst, MeanModel, ConfEst, ConfModel)
inputs = Input(shape=(5,))
MeanEst = (Dense(30, activation='tanh'))(inputs)
MeanEst = (Dropout(0.1))(MeanEst)
MeanEst = (Dense(30, activation='tanh'))(MeanEst)
MeanEst = (Dropout(0.1))(MeanEst)
MeanEst = (Dense(30, activation='tanh'))(MeanEst)
MeanEst = (Dropout(0.1))(MeanEst)
MeanEst = (Dense(30, activation='tanh'))(MeanEst)
MeanEst = (Dropout(0.1))(MeanEst)
MeanEst = (Dense(1, activation='linear'))(MeanEst)
MeanModel = Model(inputs=[inputs], outputs=MeanEst)

ConfEst= (Dense(30, activation='tanh'))(inputs)
ConfEst = (Dropout(0.1))(ConfEst)
ConfEst= (Dense(30, activation='tanh'))(ConfEst)
ConfEst = (Dropout(0.1))(ConfEst)
ConfEst= (Dense(30, activation='tanh'))(ConfEst)
ConfEst = (Dropout(0.1))(ConfEst)
ConfEst= (Dense(30, activation='tanh'))(ConfEst)
ConfEst = (Dropout(0.1))(ConfEst)
ConfEst= (Dense(1, activation='relu'))(ConfEst)
ConfModel = Model(inputs=[inputs], outputs=ConfEst)
#how can this give me a confidence output? the only thing that appears to define a confidence is the relu
#on the last layer...think about this.
#CombinedSub = Concatenate(axis=-1)([MeanEst, ConfEst])
CombinedSub = Concatenate(axis=-1)([MeanModel(inputs), ConfModel(inputs)])

CombinedModel = Model(inputs=[inputs], outputs=CombinedSub)
#plot_model(CombinedModel,to_file='demo.png',show_shapes=True)

In [ ]:
#CombinedModel.save_weights('modelsIterate/TrainingBoth_0000.hdf5')

In [ ]:
CombinedModel.summary()

## First training iteration

In [ ]:
CheckPoint = ModelCheckpoint('models_GALAXIA_nonn/TrainingMean_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR = ReduceLROnPlateau(patience=10, min_lr=1e-5, verbose=True)
CSV_logger = CSVLogger(filename='logs/training_mean.log', separator=',', append=False)

mycallbacks = [CheckPoint, ES, RLR, CSV_logger]
#mycallbacks = [ES, RLR]

In [ ]:
ConfModel.trainable = False
MeanModel.trainable = True
CombinedModel.compile(loss=ConstantLikelihoodLossFunction,
                      optimizer='adam'
                     )
history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Mean Model (Constant Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
CheckPoint2 = ModelCheckpoint('models_GALAXIA_nonn/TrainingErrorBars_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES2 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR2 = ReduceLROnPlateau(patience=10, min_lr=1e-5)
CSV_logger2 = CSVLogger(filename='logs/training_errorbars.log', separator=',', append=False)

mycallbacks2 = [CheckPoint2, ES2, RLR2, CSV_logger2]
#mycallbacks2 = [ES2, RLR2]

In [ ]:
ConfModel.trainable = True
MeanModel.trainable = False
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks2
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Confidence Model (Likelihood Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

## Second Training Iteration

In [ ]:
CheckPoint4 = ModelCheckpoint('models_GALAXIA_nonn/TrainingMean2_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES4 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR4 = ReduceLROnPlateau(patience=10, min_lr=1e-5, verbose=True)
CSV_logger4 = CSVLogger(filename='logs/training_mean2.log', separator=',', append=False)

mycallbacks4 = [CheckPoint4, ES4, RLR4, CSV_logger4]
#mycallbacks4 = [ES4, RLR4]

In [ ]:
ConfModel.trainable = False
MeanModel.trainable = True
CombinedModel.compile(loss=ConstantLikelihoodLossFunction,
                      optimizer='adam'
                     )
history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks4
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Mean Model (2nd Train) (Constant Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
CheckPoint5 = ModelCheckpoint('models_GALAXIA_nonn/TrainingErrorBars2_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES5 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR5 = ReduceLROnPlateau(patience=10, min_lr=1e-5)
CSV_logger5 = CSVLogger(filename='logs/training_errorbars2.log', separator=',', append=False)

mycallbacks5 = [CheckPoint5, ES5, RLR5, CSV_logger5]
#mycallbacks5 = [ES5, RLR5]

In [ ]:
ConfModel.trainable = True
MeanModel.trainable = False
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks5
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Confidence Model (2nd Train) (Likelihood Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

## Third Training Iteration

In [ ]:
CheckPoint6 = ModelCheckpoint('models_GALAXIA_nonn/TrainingMean3_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES4 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR4 = ReduceLROnPlateau(patience=10, min_lr=1e-5, verbose=True)
CSV_logger6 = CSVLogger(filename='logs/training_mean3.log', separator=',', append=False)

mycallbacks6 = [CheckPoint6, ES4, RLR4, CSV_logger6]
#mycallbacks4 = [ES4, RLR4]

In [ ]:
ConfModel.trainable = False
MeanModel.trainable = True
CombinedModel.compile(loss=ConstantLikelihoodLossFunction,
                      optimizer='adam'
                     )
history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks6
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Mean Model (3rd Train) (Constant Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
CheckPoint7 = ModelCheckpoint('models_GALAXIA_nonn/TrainingErrorBars3_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES5 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR5 = ReduceLROnPlateau(patience=10, min_lr=1e-5)
CSV_logger7 = CSVLogger(filename='logs/training_errorbars3.log', separator=',', append=False)

mycallbacks7 = [CheckPoint7, ES5, RLR5, CSV_logger7]
#mycallbacks5 = [ES5, RLR5]

In [ ]:
ConfModel.trainable = True
MeanModel.trainable = False
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks7,
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Confidence Model (3rd Train) (Likelihood Loss Function)')
plt.legend(['Train', 'Validation'])
plt.show()

## Train Both

In [ ]:
CheckPoint3 = ModelCheckpoint('models_GALAXIA_nonn/TrainingBoth_{epoch:04d}.hdf5',
                             verbose=0,
                             save_best_only=False
                            )
ES3 = EarlyStopping(patience=40, verbose=True, restore_best_weights=True)
RLR3 = ReduceLROnPlateau(patience=10, min_lr=1e-5)
CSV_logger3 = CSVLogger(filename='logs/training_both.log', separator=',', append=False)

mycallbacks3 = [CheckPoint3, ES3, RLR3, CSV_logger3]
#mycallbacks3 = [ES3, RLR3]

In [ ]:
ConfModel.trainable = True
MeanModel.trainable = True
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks3
                 )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
#plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Both Models (Likelihood Loss Function)')
plt.legend(['Train', 'Validation'])
plt.savefig('Loss_Plot_Iterate.png')
plt.show()

In [ ]:
 def plot_validation(i,type1):   
    y_low = -700
    y_high = 700
    plt.figure(figsize=(12, 12))
    gs0 = gridspec.GridSpec(3, 3, hspace=0.3, wspace=0.3)

    from matplotlib.colors import LogNorm
    plt.subplot(gs0[3])
    plt.hist2d( test_preds[:,1],test_preds[:,0], bins=40,norm = LogNorm())
    clb1 = plt.colorbar()
    clb1.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel('$\sigma$',labelpad=-10)

    plt.subplot(gs0[4])
    sc = plt.scatter(y_val[:,0], test_preds[:,0], c =test_preds[:,1], label = 'data',marker = '.',cmap=plt.cm.YlOrRd)
    x1 = np.linspace(y_low,y_high,1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.xlim([y_low, y_high])
    plt.ylim([y_low,y_high])
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb2 = plt.colorbar(sc)
    clb2.set_label('$\sigma$', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.clim(0,750)

    plt.subplot(gs0[2])
    hb = plt.hexbin(y_val[:,0], test_preds[:,0],gridsize=80, norm = LogNorm(), extent=[-150, 150, -150, 150])
    x1 = np.linspace(np.min(y_val[:,0]),np.max(y_val[:,0]),1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb3 = plt.colorbar(hb)
    clb3.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)


    plt.subplot(gs0[5])
    sc = plt.scatter(y_val[:,0], test_preds[:,0], c =test_preds[:,1], label = 'data',marker = '.',cmap=plt.cm.YlOrRd)
    x1 = np.linspace(y_low,y_high,1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.xlim([y_low, y_high])
    plt.ylim([y_low,y_high])
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb4 = plt.colorbar(sc)
    clb4.set_label('$\sigma$\n(saturated)', labelpad=-25, y=1.15, rotation=0,fontsize=10)
    plt.clim(0,200)

    plt.subplot(gs0[6])
    dist_hist = np.divide(np.ones_like(X_test_save[:,2]),X_val_save[:,2]*1000)
    plt.hist2d(dist_hist, test_preds_2[:,1], bins=40, norm = LogNorm())
    clb5 = plt.colorbar()
    clb5.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.xlabel('Distance',fontsize = 12)
    plt.ylabel('$\sigma$',labelpad=-5)

    plt.subplot(gs0[1])
    plotrange = np.linspace(-5,5,1000)
    diff_hist = np.divide(np.subtract(test_preds[:,0],y_val[:,0]),test_preds[:,1])
    mean_diffs, mean_stds = np.mean(diff_hist), np.std(diff_hist)
    plt.hist(diff_hist,bins=20, range=(-5,5), histtype='step',color = 'lightseagreen',linewidth = 1.5, density = True)
    plt.plot(plotrange, norm.pdf(plotrange, mean_diffs, mean_stds),color = 'darkorange', linestyle = '--', linewidth = 2.5,label = 'normal fit')
    plt.yscale('log')
    plt.legend(loc = "upper right",prop={'size': 8})
    plt.xlabel(r'$(v_{\rm{los}}^{\rm{pred}} - v_{\rm{los}}^{\rm{meas}})/\sigma$',labelpad=-5)
    
    plt.subplot(gs0[7])
    diff_hist_err = np.subtract(test_preds[:,0],y_val[:,0])
    plt.hist2d(test_preds[:,1],diff_hist_err, bins=40,norm = LogNorm())
    clb6 = plt.colorbar()
    clb6.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.plot(x1,y1,'k--')
    plt.ylabel(r'$(v_{\rm{los}}^{\rm{pred}} - v_{\rm{los}}^{\rm{meas}})$',labelpad=-5)
    plt.xlabel('$\sigma$',labelpad=-5)

    plt.subplot(gs0[0])
    plt.hist(y_val[:,0], bins=25, range=(y_low,y_high), histtype='step', edgecolor = 'skyblue', color= 'skyblue', fill = True, label = 'val' )
    plt.hist(test_preds[:,0], bins=25, range=(y_low,y_high), histtype='step',color = 'blue', label = 'predicted')
    plt.xlabel(r'$v_{\rm{los}}$', labelpad =-2)
    plt.title('Validation set, Train '+type1+', Epoch '+str(i)+', 20 bins',fontsize=14)
    plt.yscale('log')
    plt.legend(loc = "upper right",prop={'size': 10})
    
    plt.savefig('plots'+type1+'/'+'train'+type1+'_'+str(i)+'.png')
    clb1.remove()
    clb2.remove()
    clb3.remove()
    clb4.remove()
    clb5.remove()
    clb6.remove()

#### Plotting First Training

In [ ]:
traininmean = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingMean_')]
traininmean = sorted(traininmean)[1::10]

In [ ]:
%%capture
for i, mfile in enumerate(traininmean):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i*10,'Mean1');

In [ ]:
traininerror = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingErrorBars_')]
traininerror = sorted(traininerror)[0::20]

In [ ]:
%%capture
for i_err, mfile_err in enumerate(traininerror):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile_err)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i_err*20,'Error1');

#### Plotting Second Training

In [ ]:
traininmean2 = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingMean2')]
traininmean2 = sorted(traininmean2)[0::10]

In [ ]:
%%capture
for i, mfile in enumerate(traininmean2):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i*10,'Mean2');

In [ ]:
traininerror2 = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingErrorBars2')]
traininerror2 = sorted(traininerror2)[0::20]

In [ ]:
%%capture
for i_err, mfile_err in enumerate(traininerror2):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile_err)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i_err*20,'Error2');

#### Plotting Third Training

In [ ]:
traininmean3 = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingMean3')]
traininmean3 = sorted(traininmean3)[0::10]

In [ ]:
%%capture
for i, mfile in enumerate(traininmean3):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i*10,'Mean3');

In [ ]:
traininerror3 = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingErrorBars3')]
traininerror3 = sorted(traininerror3)[0::20]

In [ ]:
%%capture
for i_err, mfile_err in enumerate(traininerror3):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile_err)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i_err*20,'Error3');

#### Plot results of training both

In [ ]:
traininboth = [file for file in os.listdir('models_GALAXIA_nonn/')
               if file.startswith('TrainingBoth')]
traininboth = sorted(traininboth)[1::10]


In [ ]:
%%capture
for i_both, mfile_both in enumerate(traininboth):
    CombinedModel.load_weights('models_GALAXIA_nonn/' + mfile_both)
    
    test_preds = CombinedModel.predict(X_val)

    
    plot_validation(i_both*10,'Both');

In [ ]:
print(np.size(y_test))

### Evaluate the Test Set

In [ ]:
#CombinedModel.load_weights('G_train_2it_50k/models_GALAXIA_nonn/' + 'TrainingBoth_0109.hdf5')
test_preds_2 = CombinedModel.predict(X_test)
y_test = SS_rv.inverse_transform(y_test)
test_preds_2[:,0] = SS_rv.inverse_transform(test_preds_2[:,0])
test_preds_2[:,1] = SS_rv.inverse_transform(test_preds_2[:,1])
y_test = y_test*Y_train_max
test_preds_2[:,0] = test_preds_2[:,0]*Y_train_max
test_preds_2[:,1] = test_preds_2[:,1]*Y_train_max

y_low = -250
y_high = 250
plt.figure()
plt.hist(y_test, bins=50, range=(y_low,y_high), histtype='step', edgecolor = 'skyblue', color= 'skyblue', fill = True, label = 'test', density = True)
plt.hist(test_preds_2[:,0], bins=50, range=(y_low,y_high), histtype='step',color = 'blue', label = 'predicted', density = True)
plt.xlabel('Vr')
plt.ylabel('Stars')
plt.title('Test set - simulated data')
#plt.yscale('log')
plt.legend(loc = "upper right")
plt.show()

## Going to use Monte Carlo simulations to predict errorbars. 

In [ ]:
mc_vr_pred_list = []
bin_values_list = []
min_array = []
max_array = []
mean_array = []
for mc_i in range(0,1000):
    #print(mc_i)
    mc_vr_pred = []
    for star_i in range(0,np.size(y_test)):
        #mc_vr_pred.append(np.mean([np.random.normal(test_preds_2[star_i,0],test_preds_2[star_i,1]) for _ in range(1000)]))
        mc_vr_pred.append(np.random.normal(test_preds_2[star_i,0],test_preds_2[star_i,1]))
    mc_vr_pred_list.append(mc_vr_pred)
    n, bins, patches = plt.hist(mc_vr_pred, bins=20, range=(y_low,y_high), histtype='step')
    plt.xlabel('Vr Monte Carlo')
    plt.yscale('log')
    bin_values_list.append(n)
bin_values_list_arr = np.array(bin_values_list)    
max_array = bin_values_list_arr.max(axis=0)
min_array = bin_values_list_arr.min(axis=0) 
mean_array = bin_values_list_arr.mean(axis=0)
y_error = [min_array,max_array]
print(np.shape(y_error))

In [ ]:
plt.figure()
hist_test, bins_test, patches_test = plt.hist(y_test, bins=20, range=(y_low,y_high), histtype='step', edgecolor = 'skyblue', color= 'skyblue', fill = True, label = 'test' , zorder = 0)
#plt.hist(bins_test[:-1], bins_test, weights=max_array,color = 'green')
#plt.hist(bins_test[:-1], bins_test, weights=min_array,color = 'yellow')
bin_centers_test = (bins_test[1:]+bins_test[:-1])/2
plt.fill_between(bin_centers_test,min_array, max_array,label = 'MC spread',color = 'orange', zorder = 10)

plt.hist(test_preds_2[:,0], bins=20, range=(y_low,y_high), histtype='step',color = 'blue', label = 'predicted', zorder = 20)
print(y_low,y_high)
plt.xlabel(r'$v_{\rm{los}}$')
plt.title('Test set - simulated data')
plt.yscale('log')
plt.legend(loc = "upper right",prop={'size': 10})
plt.savefig('MC_100trials_8000stars_noavg.png')
plt.show()

In [ ]:
 def plot_test():   
    y_low = -200
    y_high = 200
    plt.figure(figsize=(12, 12))
    gs0 = gridspec.GridSpec(3, 3, hspace=0.3, wspace=0.3)

    from matplotlib.colors import LogNorm
    plt.subplot(gs0[3])
    plt.hist2d(test_preds_2[:,1],test_preds_2[:,0], bins=40,norm = LogNorm())
    clb1 = plt.colorbar()
    clb1.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel('$\sigma$',labelpad=-5)
    

    plt.subplot(gs0[4])
    sc = plt.scatter(y_test[:,0], test_preds_2[:,0], c =test_preds_2[:,1], label = 'data',marker = '.',cmap=plt.cm.YlOrRd)
    x1 = np.linspace(y_low,y_high,1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.xlim([y_low, y_high])
    plt.ylim([y_low,y_high])
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb2 = plt.colorbar(sc)
    clb2.set_label('$\sigma$', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.clim(0,750)

    plt.subplot(gs0[2])
    hb = plt.hexbin(y_test[:,0], test_preds_2[:,0],gridsize=80, norm = LogNorm(), extent = [-150,150,-150,150])
    x1 = np.linspace(np.min(test_preds_2[:,0]),np.max(test_preds_2[:,0]),1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb3 = plt.colorbar(hb)
    clb3.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)


    plt.subplot(gs0[5])
    sc = plt.scatter(y_test[:,0], test_preds_2[:,0], c =test_preds_2[:,1], label = 'data',marker = '.',cmap=plt.cm.YlOrRd)
    x1 = np.linspace(y_low,y_high,1000)
    y1 = x1
    plt.plot(x1,y1,'k--')
    plt.xlim([y_low, y_high])
    plt.ylim([y_low,y_high])
    plt.ylabel(r'$v_{\rm{los}}^{\rm{pred}}$',labelpad=-10)
    plt.xlabel(r'$v_{\rm{los}}^{\rm{meas}}$')
    clb4 = plt.colorbar(sc)
    clb4.set_label('$\sigma$\n(saturated)', labelpad=-25, y=1.15, rotation=0,fontsize=10)
    plt.clim(0,200)

    plt.subplot(gs0[6])
    dist_hist = np.divide(np.ones_like(X_test_save[:,2]),X_test_save[:,2])
    plt.hist2d(dist_hist, test_preds_2[:,1], bins=40, norm = LogNorm())
    clb5 = plt.colorbar()
    clb5.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.xlabel('Distance (kpc)',fontsize = 12)
    plt.ylabel('$\sigma$',labelpad=-5)

    plt.subplot(gs0[1])
    plotrange = np.linspace(-5,5,1000)
    diff_hist = np.divide(np.subtract(test_preds_2[:,0],y_test[:,0]),test_preds_2[:,1])
    mean_diffs, mean_stds = np.mean(diff_hist), np.std(diff_hist)
    plt.hist(diff_hist,bins=20, range=(-5,5), histtype='step',color = 'lightseagreen',linewidth = 1.5, density = True)
    plt.plot(plotrange, norm.pdf(plotrange, mean_diffs, mean_stds),color = 'darkorange', linestyle = '--', linewidth = 2.5,label = 'normal fit')
    plt.yscale('log')
    plt.legend(loc = "upper right",prop={'size': 8})
    plt.xlabel(r'$(v_{\rm{los}}^{\rm{pred}} - v_{\rm{los}}^{\rm{meas}})/\sigma$',labelpad=-5)
    
    plt.subplot(gs0[7])
    plt.hist2d(X_test_save[:,0],test_preds_2[:,1], bins=40,norm = LogNorm())
    clb6 = plt.colorbar()
    clb6.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.plot(x1,y1,'k--')
    plt.xlabel(r'$l$',labelpad=-3)
    plt.ylabel('$\sigma$',labelpad=-5)
    
    plt.subplot(gs0[8])
    plt.hist2d(X_test_save[:,1],test_preds_2[:,1], bins=40,norm = LogNorm())
    clb6 = plt.colorbar()
    clb6.set_label('Density', labelpad=-25, y=1.08, rotation=0,fontsize=10)
    plt.plot(x1,y1,'k--')
    plt.xlabel(r'$b$',labelpad=-3)
    plt.ylabel('$\sigma$',labelpad=-5)

    plt.subplot(gs0[0])
    plt.hist(y_test, bins=50, range=(y_low,y_high), histtype='step', edgecolor = 'skyblue', color= 'skyblue', fill = True, label = 'test', density = True )
    plt.hist(test_preds_2[:,0], bins=50, range=(y_low,y_high), histtype='step',color = 'blue', label = 'predicted', density = True)
    plt.xlabel(r'$v_{\rm{los}}$', labelpad =-2)
    plt.title('Test set - simulated data, 50 bins',fontsize=14)
    #plt.yscale('log')
    plt.legend(loc = "upper right",prop={'size': 10})
    plt.savefig('G_plots_test_2it_500k_tanh_NORM1st_try2.png')
    #plt.savefig('plots_error/trainerror_'+str(i)+'.png')
    clb1.remove()
    clb2.remove()
    clb3.remove()
    clb4.remove()
    clb5.remove()
    clb6.remove()

In [ ]:
plot_test()

##### Going to try to find original indices of points with low error in train, validation, and test & retrain without those

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#import seaborn as sns
 
# Make the plot
fig = plt.figure(figsize=(12, 12))
ax = fig.gca(projection='3d')
ax.plot_trisurf(X_test_save[:,0], X_test_save[:,1], test_preds_2[:,1], cmap=plt.cm.YlOrRd, linewidth=0.2)
ax.set_xlabel('$l$')
ax.set_ylabel('$b$')
ax.set_zlabel('$\sigma$',rotation=0)
ax.view_init(azim=-30, elev=50)
surf=ax.plot_trisurf(X_test_save[:,0], X_test_save[:,1], test_preds_2[:,1], cmap=plt.cm.YlOrRd, linewidth=0.2)
fig.colorbar( surf, shrink=0.5, aspect=5, label = '$\sigma$')
plt.show()
 
# to Add a color bar which maps values to colors.
surf=ax.plot_trisurf(X_test_save[:,0], X_test_save[:,1], test_preds_2[:,1], cmap=plt.cm.YlOrRd, linewidth=0.2)
fig.colorbar( surf, shrink=0.5, aspect=5)
plt.show()

 
# Other palette
ax.plot_trisurf(X_test_save[:,0], X_test_save[:,1], test_preds_2[:,1], cmap=plt.cm.YlOrRd, linewidth=0.01)
plt.show()
 

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

data=np.vstack((X_test_save[:,0], X_test_save[:,1], test_preds_2[:,1]))
data = np.transpose(data)
values = data.T
kde = stats.gaussian_kde(values)
density = kde(values)



In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.gca(projection='3d')
x, y, z = values
surf1 = ax.scatter(x, y, z, c=density)
ax.set_xlabel('$l$')
ax.set_ylabel('$b$')
ax.set_zlabel('$\sigma$',rotation=0)
ax.view_init(azim=-60, elev=10)
fig.colorbar( surf1, shrink=0.5, aspect=5, label = 'density')
plt.show()

In [ ]:
print(np.max(test_preds_2[:,1]))

 # min_cl = np.nanmin(code_lengths)
  #  min_k = Ks[code_lengths.index(min_cl)] 
list_pmra_test_err_lt65 = []   
list_rv_test_err_lt65 = [y_test[i,0] for i in range(len(test_preds_2[:,1])) if test_preds_2[i,1] < 90]
print(len(list_rv_test_err_lt65))
#now need indices of these values in data
indices = []
for i in range(len(list_rv_test_err_lt65)):
    indices.append(data[data['radial_velocity']==(list_rv_test_err_lt65[i])].index[0])

In [ ]:
CombinedModel.load_weights('models/' + 'TrainingBoth_0138.hdf5')
test_preds_train = CombinedModel.predict(X_train)

list_rv_train_err_lt65 = [y_train[i,0] for i in range(len(test_preds_train[:,1])) if test_preds_train[i,1] < 90]
for i in range(len(list_rv_train_err_lt65)):
    indices.append(data[data['radial_velocity']==(list_rv_train_err_lt65[i])].index[0])

In [ ]:
test_preds_val = CombinedModel.predict(X_val)

list_rv_val_err_lt65 = [y_val[i,0] for i in range(len(test_preds_val[:,1])) if test_preds_val[i,1] < 90]
for i in range(len(list_rv_val_err_lt65)):
    indices.append(data[data['radial_velocity']==(list_rv_val_err_lt65[i])].index[0])

In [ ]:
np.save('data_indices_error_lt100_50k',indices)

In [ ]:
print(np.shape(indices))